In [1]:
using StatisticalRethinking, Distributed, JLD, LinearAlgebra
using Mamba

Data

In [2]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [3]:
df2 = filter(row -> row[:age] >= 18, df);

Input data for Mamba

In [4]:
data = Dict(
  :x => convert(Array{Float64,1}, df2[:weight]),
  :y => convert(Array{Float64,1}, df2[:height])
);

Log-transformed Posterior(b0, b1, log(s2)) + Constant and Gradient Vector

In [5]:
logfgrad = function(x::DenseVector)
  b0 = x[1]
  b1 = x[2]
  logs2 = x[3]
  r = data[:y] .- b0 .- b1 .* data[:x]
  logf = (-0.5 * length(data[:y]) - 0.001) * logs2 -
           (0.5 * dot(r, r) + 0.001) / exp(logs2) -
           0.5 * b0^2 / 1000 - 0.5 * b1^2 / 1000
  grad = [
    sum(r) / exp(logs2) - b0 / 1000,
    sum(data[:x] .* r) / exp(logs2) - b1 / 1000,
    -0.5 * length(data[:y]) - 0.001 + (0.5 * dot(r, r) + 0.001) / exp(logs2)
  ]
  logf, grad
end

# MCMC Simulation with No-U-Turn Sampling

n = 5000
burnin = 1000
sim = Mamba.Chains(n, 3, start = (burnin + 1), names = ["b0", "b1", "s2"])
theta = NUTSVariate([0.0, 0.0, 0.0], logfgrad)
for i in 1:n
  sample!(theta, adapt = (i <= burnin))
  if i > burnin
    sim[i, :, 1] = [theta[1:2]; exp(theta[3])]
  end
end

Summarize draws

In [6]:
describe(sim)#-

Iterations = 1001:5000
Thinning interval = 1
Chains = 1
Samples per chain = 4000

Empirical Posterior Estimates:
       Mean         SD       Naive SE       MCSE         ESS   
b0 113.5392677 1.842859364 0.0291381650 0.0819977589  505.10355
b1   0.9125045 0.040720256 0.0006438438 0.0018125294  504.72042
s2  25.9505767 1.981245312 0.0313262389 0.0468760788 1786.38043

Quantiles:
       2.5%        25.0%       50.0%       75.0%       97.5%   
b0 109.8882969 112.29198982 113.5362340 114.7910701 117.0112583
b1   0.8358232   0.88441987   0.9131553   0.9402706   0.9926724
s2  22.3522341  24.60865040  25.8522463  27.2616595  30.0620662



*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*